In [ ]:
import pandas as pd
import re
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import (
    wordnet,
    stopwords
)
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
df=pd.read_csv('/content/suns-bucks-G6.csv', sep=',', engine='python', error_bad_lines=False)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Parte 2
df1=df

In [ ]:
#preprocess text in tweets by removing links, @UserNames, blank spaces, etc.
def preprocessing_text(table):
    #put everythin in lowercase
    table['text'] = table['text'].str.lower()
    #Replace rt indicating that was a retweet
    table['text'] = table['text'].str.replace('rt', '')
    #Replace occurences of mentioning @UserNames
    table['text'] = table['text'].replace(r'@\w+', '', regex=True)
    #Replace links contained in the tweet
    table['text'] = table['text'].replace(r'http\S+', '', regex=True)
    table['text'] = table['text'].replace(r'www.[^ ]+', '', regex=True)
    #remove numbers
    table['text'] = table['text'].replace(r'[0-9]+', '', regex=True)
    #replace special characters and puntuation marks
    table['text'] = table['text'].replace(r'[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~]', '', regex=True)
    return table

In [ ]:
#Replace elongated words by identifying those repeated characters and then remove them and compare the new word with the english lexicon
def in_dict(word):
    if wordnet.synsets(word):
        #if the word is in the dictionary, we'll return True
        return True

def replace_elongated_word(word):
    regex = r'(\w*)(\w+)\2(\w*)'
    repl = r'\1\2\3'    
    if in_dict(word):
        return word
    new_word = re.sub(regex, repl, word)
    if new_word != word:
        return replace_elongated_word(new_word)
    else:
        return new_word

def detect_elongated_words(row):
    regexrep = r'(\w*)(\w+)(\2)(\w*)'
    words = [''.join(i) for i in re.findall(regexrep, row)]
    for word in words:
        if not in_dict(word):
            row = re.sub(word, replace_elongated_word(word), row)
    return row

In [ ]:
def stop_words(table):
    #We need to remove the stop words
    stop_words_list = stopwords.words('english')
    table['text'] = table['text'].str.lower()
    table['text'] = table['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words_list)]))
    return table

In [ ]:
def replace_antonyms(word):
    #We get all the lemma for the word
    for syn in wordnet.synsets(word): 
        for lemma in syn.lemmas(): 
            #if the lemma is an antonyms of the word
            if lemma.antonyms(): 
                #we return the antonym
                return lemma.antonyms()[0].name()
    return word
            
def handling_negation(row):
    #Tokenize the row
    words = word_tokenize(row)
    speach_tags = ['JJ', 'JJR', 'JJS', 'NN', 'VB', 'VBD', 'VBG', 'VBN', 'VBP']
    #We obtain the type of words that we have in the text, we use the pos_tag function
    tags = nltk.pos_tag(words)
    #Now we ask if we found a negation in the words
    tags_2 = ''
    if "n't" in words and "not" in words:
        tags_2 = tags[min(words.index("n't"), words.index("not")):]
        words_2 = words[min(words.index("n't"), words.index("not")):]
        words = words[:(min(words.index("n't"), words.index("not")))+1]
    elif "n't" in words:
        tags_2 = tags[words.index("n't"):]
        words_2 = words[words.index("n't"):] 
        words = words[:words.index("n't")+1]
    elif "not" in words:
        tags_2 = tags[words.index("not"):]
        words_2 = words[words.index("not"):]
        words = words[:words.index("not")+1] 
        
    for index, word_tag in enumerate(tags_2):
        if word_tag[1] in speach_tags:
            words = words+[replace_antonyms(word_tag[0])]+words_2[index+2:]
            break
            
    return ' '.join(words)

In [ ]:
def cleaning_table(table):
    #This function will process all the required cleaning for the text in our tweets
    table = preprocessing_text(table)
    table['text'] = table['text'].apply(lambda x: detect_elongated_words(x))
    table['text'] = table['text'].apply(lambda x: handling_negation(x))
    table = stop_words(table)
    return table

In [ ]:
df1=cleaning_table(df1)

In [ ]:
df1.head()

,Unnamed: 0,_id,id,user_location,language,timestamp,text,retweet_count,user_followers,geo,coordinates,retweeted,is_a_retweet,user_mentions,entities_hashtags,in_reply_to_status_id_str,in_reply_to_user_id_str,retweeted_status_id,retweeted_status__user_id,retweeted_status_user_name,id_str,screen_name
0,0,60f74336b712538547cbf0fc,1417600743033360384,AZ,en,2021-07-20 23:42:09,𝐍𝐁𝐀 𝐅𝐈𝐍𝐀𝐋𝐒 𝐆𝐀𝐌𝐄 𝟔 📺 try free 📻 𝐋𝐎𝐂𝐊𝐄𝐃 𝐈𝐍 ralyt...,0,1537,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'Suns', 'name': 'Phoenix Suns...","[{'text': 'RallyTheValley', 'indices': [120, 1...",NaN,NaN,1.417543e+18,1.848111e+07,Phoenix Suns,1146370740,boi_guapo
1,1,60f74338b712538547cbf0fd,1417600749182365700,"Stockholm, Sverige",en,2021-07-20 23:42:11,breaking news nba ref scott foster given chris...,0,1036,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'davidalangrier', 'name': 'Da...","[{'text': 'NBAFinals', 'indices': [120, 130]}]",NaN,NaN,1.417601e+18,1.814409e+07,David Alan Grier,1061255281341054977,Ahmedberhan1
2,2,60f7433ab712538547cbf0fe,1417600757872959492,By the Rich People,en,2021-07-20 23:42:13,k streams‼️ supoers beginning thank holding us...,0,2485,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'wildcardsprts', 'name': 'Wil...",[],NaN,NaN,1.417596e+18,1.142281e+18,WildCard Sports,239012690,Pastor_KevyKev
3,3,60f7433bb712538547cbf0ff,1417600761500999681,"Charlotte, NC",en,2021-07-20 23:42:13,many postit notes,0,174837,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>",[],[],NaN,NaN,NaN,NaN,NaN,21362008,tomhaberstroh
4,4,60f7433bb712538547cbf100,1417600763707154435,NaN,en,2021-07-20 23:42:14,nba 🏀 july th ayton points like tailing🔥 betin...,0,664,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'JohnnyPropz', 'name': 'Johnn...","[{'text': 'BettingExpert', 'indices': [82, 96]...",NaN,NaN,1.417601e+18,1.356723e+18,Johnny Props,1281401105398796288,JohnnyLockz


In [ ]:
!pip install vaderSentiment

In [ ]:
!pip install twython

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
df1['sentiment'] = df1['text'].apply(analyzer.polarity_scores)

In [ ]:
df1['compound']  = df1['sentiment'].apply(lambda score_dict: score_dict['compound'])

In [ ]:
df1.head()

,Unnamed: 0,_id,id,user_location,language,timestamp,text,retweet_count,user_followers,geo,coordinates,retweeted,is_a_retweet,user_mentions,entities_hashtags,in_reply_to_status_id_str,in_reply_to_user_id_str,retweeted_status_id,retweeted_status__user_id,retweeted_status_user_name,id_str,screen_name,sentiment,compound
0,0,60f74336b712538547cbf0fc,1417600743033360384,AZ,en,2021-07-20 23:42:09,𝐍𝐁𝐀 𝐅𝐈𝐍𝐀𝐋𝐒 𝐆𝐀𝐌𝐄 𝟔 📺 try free 📻 𝐋𝐎𝐂𝐊𝐄𝐃 𝐈𝐍 ralyt...,0,1537,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'Suns', 'name': 'Phoenix Suns...","[{'text': 'RallyTheValley', 'indices': [120, 1...",NaN,NaN,1.417543e+18,1.848111e+07,Phoenix Suns,1146370740,boi_guapo,"{'neg': 0.0, 'neu': 0.708, 'pos': 0.292, 'comp...",0.5106
1,1,60f74338b712538547cbf0fd,1417600749182365700,"Stockholm, Sverige",en,2021-07-20 23:42:11,breaking news nba ref scott foster given chris...,0,1036,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'davidalangrier', 'name': 'Da...","[{'text': 'NBAFinals', 'indices': [120, 130]}]",NaN,NaN,1.417601e+18,1.814409e+07,David Alan Grier,1061255281341054977,Ahmedberhan1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,2,60f7433ab712538547cbf0fe,1417600757872959492,By the Rich People,en,2021-07-20 23:42:13,k streams‼️ supoers beginning thank holding us...,0,2485,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'wildcardsprts', 'name': 'Wil...",[],NaN,NaN,1.417596e+18,1.142281e+18,WildCard Sports,239012690,Pastor_KevyKev,"{'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'comp...",0.3612
3,3,60f7433bb712538547cbf0ff,1417600761500999681,"Charlotte, NC",en,2021-07-20 23:42:13,many postit notes,0,174837,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>",[],[],NaN,NaN,NaN,NaN,NaN,21362008,tomhaberstroh,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
4,4,60f7433bb712538547cbf100,1417600763707154435,NaN,en,2021-07-20 23:42:14,nba 🏀 july th ayton points like tailing🔥 betin...,0,664,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'JohnnyPropz', 'name': 'Johnn...","[{'text': 'BettingExpert', 'indices': [82, 96]...",NaN,NaN,1.417601e+18,1.356723e+18,Johnny Props,1281401105398796288,JohnnyLockz,"{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound...",0.3612


In [ ]:
!pip install NRCLex

In [ ]:
from nrclex import NRCLex
def get_emotions(sentence):
  text_object = NRCLex(sentence)
  return text_object.raw_emotion_scores

In [ ]:
df1['emotion'] = df1.text.apply(get_emotions)
df1.head()

,Unnamed: 0,_id,id,user_location,language,timestamp,text,retweet_count,user_followers,geo,coordinates,retweeted,is_a_retweet,user_mentions,entities_hashtags,in_reply_to_status_id_str,in_reply_to_user_id_str,retweeted_status_id,retweeted_status__user_id,retweeted_status_user_name,id_str,screen_name,sentiment,compound,emotion
0,0,60f74336b712538547cbf0fc,1417600743033360384,AZ,en,2021-07-20 23:42:09,𝐍𝐁𝐀 𝐅𝐈𝐍𝐀𝐋𝐒 𝐆𝐀𝐌𝐄 𝟔 📺 try free 📻 𝐋𝐎𝐂𝐊𝐄𝐃 𝐈𝐍 ralyt...,0,1537,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'Suns', 'name': 'Phoenix Suns...","[{'text': 'RallyTheValley', 'indices': [120, 1...",NaN,NaN,1.417543e+18,1.848111e+07,Phoenix Suns,1146370740,boi_guapo,"{'neg': 0.0, 'neu': 0.708, 'pos': 0.292, 'comp...",0.5106,{}
1,1,60f74338b712538547cbf0fd,1417600749182365700,"Stockholm, Sverige",en,2021-07-20 23:42:11,breaking news nba ref scott foster given chris...,0,1036,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'davidalangrier', 'name': 'Da...","[{'text': 'NBAFinals', 'indices': [120, 130]}]",NaN,NaN,1.417601e+18,1.814409e+07,David Alan Grier,1061255281341054977,Ahmedberhan1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,{}
2,2,60f7433ab712538547cbf0fe,1417600757872959492,By the Rich People,en,2021-07-20 23:42:13,k streams‼️ supoers beginning thank holding us...,0,2485,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'wildcardsprts', 'name': 'Wil...",[],NaN,NaN,1.417596e+18,1.142281e+18,WildCard Sports,239012690,Pastor_KevyKev,"{'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'comp...",0.3612,{}
3,3,60f7433bb712538547cbf0ff,1417600761500999681,"Charlotte, NC",en,2021-07-20 23:42:13,many postit notes,0,174837,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>",[],[],NaN,NaN,NaN,NaN,NaN,21362008,tomhaberstroh,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,{}
4,4,60f7433bb712538547cbf100,1417600763707154435,NaN,en,2021-07-20 23:42:14,nba 🏀 july th ayton points like tailing🔥 betin...,0,664,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'JohnnyPropz', 'name': 'Johnn...","[{'text': 'BettingExpert', 'indices': [82, 96]...",NaN,NaN,1.417601e+18,1.356723e+18,Johnny Props,1281401105398796288,JohnnyLockz,"{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound...",0.3612,{}


In [ ]:
positive_sum=[]
for j in range(len(df1)):
  if df1.emotion[j].setdefault('positive') !=None:
    i=df1.emotion[j].setdefault('positive')
  else:
    i=0
  
  positive_sum.append(i)

anger_sum=[]
for j in range(len(df1)):
  if df1.emotion[j].setdefault('anger') !=None:
    i=df1.emotion[j].setdefault('anger')
  else:
    i=0
    
  anger_sum.append(i)

anticipation_sum=[]
for j in range(len(df1)):
  if df1.emotion[j].setdefault('anticipation') !=None:
    i=df1.emotion[j].setdefault('anticipation')
  else:
    i=0
    
  anticipation_sum.append(i)

trust_sum=[]
for j in range(len(df1)):
  if df1.emotion[j].setdefault('trust') !=None:
    i=df1.emotion[j].setdefault('trust')
  else:
    i=0
    
  trust_sum.append(i)

surprise_sum=[]
for j in range(len(df1)):
  if df1.emotion[j].setdefault('surprise') !=None:
    i=df1.emotion[j].setdefault('surprise')
  else:
    i=0
    
  surprise_sum.append(i)

negative_sum=[]
for j in range(len(df1)):
  if df1.emotion[j].setdefault('negative') !=None:
    i=df1.emotion[j].setdefault('negative')
  else:
    i=0
    
  negative_sum.append(i)

sadness_sum=[]
for j in range(len(df1)):
  if df1.emotion[j].setdefault('sadness') !=None:
    i=df1.emotion[j].setdefault('sadness')
  else:
    i=0
    
  sadness_sum.append(i)

disgust_sum=[]
for j in range(len(df1)):
  if df1.emotion[j].setdefault('disgust') !=None:
    i=df1.emotion[j].setdefault('disgust')
  else:
    i=0
    
  disgust_sum.append(i)

joy_sum=[]
for j in range(len(df1)):
  if df1.emotion[j].setdefault('joy') !=None:
    i=df1.emotion[j].setdefault('joy')
  else:
    i=0
    
  joy_sum.append(i)

fear_sum=[]
for j in range(len(df1)):
  if df1.emotion[j].setdefault('fear') !=None:
    i=df1.emotion[j].setdefault('fear')
  else:
    i=0
    
  fear_sum.append(i)

In [ ]:
df1['positive']=positive_sum
df1['anger']= anger_sum
df1['anticipation']= anticipation_sum
df1['trust']= trust_sum
df1['surprise']= surprise_sum
df1['negative']= negative_sum
df1['sadness']= sadness_sum
df1['disgust']= disgust_sum
df1['joy']= joy_sum
df1['fear']= fear_sum

In [ ]:
df1.head(50)

,Unnamed: 0,_id,id,user_location,language,timestamp,text,retweet_count,user_followers,geo,coordinates,retweeted,is_a_retweet,user_mentions,entities_hashtags,in_reply_to_status_id_str,in_reply_to_user_id_str,retweeted_status_id,retweeted_status__user_id,retweeted_status_user_name,id_str,screen_name,sentiment,compound,emotion,positive,anger,anticipation,trust,surprise,negative,sadness,disgust,joy,fear
0,0,60f74336b712538547cbf0fc,1417600743033360384,AZ,en,2021-07-20 23:42:09,𝐍𝐁𝐀 𝐅𝐈𝐍𝐀𝐋𝐒 𝐆𝐀𝐌𝐄 𝟔 📺 try free 📻 𝐋𝐎𝐂𝐊𝐄𝐃 𝐈𝐍 ralyt...,0,1537,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'Suns', 'name': 'Phoenix Suns...","[{'text': 'RallyTheValley', 'indices': [120, 1...",NaN,NaN,1.417543e+18,1.848111e+07,Phoenix Suns,1146370740,boi_guapo,"{'neg': 0.0, 'neu': 0.708, 'pos': 0.292, 'comp...",0.5106,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0
1,1,60f74338b712538547cbf0fd,1417600749182365700,"Stockholm, Sverige",en,2021-07-20 23:42:11,breaking news nba ref scott foster given chris...,0,1036,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'davidalangrier', 'name': 'Da...","[{'text': 'NBAFinals', 'indices': [120, 130]}]",NaN,NaN,1.417601e+18,1.814409e+07,David Alan Grier,1061255281341054977,Ahmedberhan1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0
2,2,60f7433ab712538547cbf0fe,1417600757872959492,By the Rich People,en,2021-07-20 23:42:13,k streams‼️ supoers beginning thank holding us...,0,2485,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'wildcardsprts', 'name': 'Wil...",[],NaN,NaN,1.417596e+18,1.142281e+18,WildCard Sports,239012690,Pastor_KevyKev,"{'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'comp...",0.3612,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0
3,3,60f7433bb712538547cbf0ff,1417600761500999681,"Charlotte, NC",en,2021-07-20 23:42:13,many postit notes,0,174837,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>",[],[],NaN,NaN,NaN,NaN,NaN,21362008,tomhaberstroh,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0
4,4,60f7433bb712538547cbf100,1417600763707154435,NaN,en,2021-07-20 23:42:14,nba 🏀 july th ayton points like tailing🔥 betin...,0,664,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'JohnnyPropz', 'name': 'Johnn...","[{'text': 'BettingExpert', 'indices': [82, 96]...",NaN,NaN,1.417601e+18,1.356723e+18,Johnny Props,1281401105398796288,JohnnyLockz,"{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound...",0.3612,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0
5,5,60f7433eb712538547cbf101,1417600777049317378,with Nayi,en,2021-07-20 23:42:17,latest kanye west rumors saying premiere new s...,0,351,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'XXL', 'name': 'XXL Magazine'...",[],NaN,NaN,1.417581e+18,1.462932e+07,XXL Magazine,2945507472,KanJayW3st,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,"{'positive': None, 'anger': None, 'anticipatio...",0,0,0,0,0,0,0,0,0,0
6,6,60f74341b712538547cbf102,1417600787216220168,NaN,en,2021-07-20 23:42:20,squad getting ready game bucksvsuns feaheder r...,0,202,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'joes_talk', 'name': 'Average...","[{'text': 'BucksVsSuns', 'indices': [57, 69]},...",NaN,NaN,1.417599e+18,1.000043e+18,Average Joe Sports Talk Podcast,1264734482487279616,dudes_couch,"{'neg': 0.0, 'neu': 0.8, 'pos': 0.2, 'compound...",0.3612,"{'anticipation': 1, 'positive': None, 'anger':...",0,0,1,0,0,0,0,0,0,0
7,7,60f74341b712538547cbf103,1417600787887366146,NaN,fr,2021-07-20 23:42:20,🏀🇺🇸 les nbafinals peuvent désigner un nouveau ...,0,35,NaN,NaN,False,"<Thread(Thread-5, started 140013339797248)>","[{'screen_name': 'ParionsSport', 'name': 'Pari...","[{'text': 'NBAFinals', '

In [ ]:
from google.colab import files
df1.to_csv('SunsBucksClean.csv')
files.download('SunsBucksClean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>